# Python & SQL: Connecting Data with SQLAlchemy

## Introduction
Welcome to tutorial on connecting Python to SQL databases using SQLAlchemy. In this guide, I'll walk you through the process of establishing connections, creating data models, and executing queries. By the end, you'll be equipped to seamlessly handle data between Python and SQL environments.

<div style="text-align:center;">
    <img src="python+sql.png" alt="python+sql" style="display:block; margin:auto;" width="750"/>
</div>


## Setting Up the Environment

For this tutorial, make sure you have the following Python packages installed:
- `pandas`
- `sqlalchemy`

You can install them using pip:

In [ ]:
!pip install pandas sqlalchemy

Import Required Libraries

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sqlalchemy import create_engine, Column, Integer, String, Float, DateTime, ForeignKey, Index, select
from sqlalchemy.orm import declarative_base, relationship, Session

## Prepare dataset

In [ ]:
# Skip this part if you have your dataframe is ready
class EcommerceDataGenerator:
    def __init__(self):
        self.categories = [('Smartphones', 2), ('Smartwatch', 2), ('Accessories', 1)]  # Adjust frequencies as needed

    def generate_customer_data(self, num_customers):
        customers = pd.DataFrame({
            'customer_id': range(1, num_customers + 1),
            'name': [f'Customer{i}' for i in range(1, num_customers + 1)],
            'email': [f'customer{i}@example.com' for i in range(1, num_customers + 1)]
        })
        return customers

    def generate_products(self, num_products):
        categories = self.categories.copy()
        products = []

        while num_products > 0:
            for category, _ in categories:
                if num_products == 0:
                    break
                products.append({'category': category})
                num_products -= 1

        prices = [self._generate_price(product['category']) for product in products]
        products = pd.DataFrame({
            'product_id': range(1, len(products) + 1),
            'name': [f'Product{j}' for j in range(1, len(products) + 1)],
            'price': prices,
            'category': [product['category'] for product in products]
        })
        return products

    def _generate_price(self, category):
        if category == 'Smartphones':
            return np.round(np.random.uniform(1500, 5000), 1)
        elif category == 'Smartwatch':
            return np.round(np.random.uniform(700, 3200), 1)
        else:
            return np.round(np.random.uniform(100, 200), 1)

    def generate_orders(self, customers, products):
        orders = []
        for customer_id in customers['customer_id']:
            num_orders = np.random.randint(1, 5)  
            for _ in range(num_orders):
                order_date = (datetime.now() - timedelta(days=np.random.randint(1, 365),
                                                      seconds=np.random.randint(0, 86400)))
                product_id = np.random.randint(1, len(products) + 1)
                orders.append({
                    'customer_id': customer_id,
                    'product_id': product_id,
                    'quantity': np.random.randint(3, 50),
                    'order_date': order_date
                })

        orders = pd.DataFrame(orders)
        return orders

    def generate_sample_data(self, num_customers, num_products):
        customers = self.generate_customer_data(num_customers)
        products = self.generate_products(num_products)
        orders = self.generate_orders(customers, products)
        return customers, products, orders

# Example usage:
ecommerce_generator = EcommerceDataGenerator()
customers, products, orders = ecommerce_generator.generate_sample_data(100, 7)

In [ ]:
customers

## Define the Database Structure

In [ ]:
# Define data models using SQLAlchemy's declarative_base
Base = declarative_base()

# Customer data model
class Customer(Base):
    __tablename__ = 'customers'
    customer_id = Column(Integer, primary_key=True)
    name = Column(String)
    email = Column(String)
    # define idx_customer_id as table index
    __table_args__ = (Index('idx_customer_id', 'customer_id'),)

# Product data model
class Product(Base):
    __tablename__ = 'products'
    product_id = Column(Integer, primary_key=True)
    name = Column(String)
    price = Column(Float)
    category = Column(String)
    # define idx_product_id as table index
    __table_args__ = (Index('idx_product_id', 'product_id'),)

# Order data model
class Order(Base):
    __tablename__ = 'orders'
    order_id = Column(Integer, primary_key=True)
    customer_id = Column(Integer, ForeignKey('customers.customer_id'))
    product_id = Column(Integer, ForeignKey('products.product_id'))
    quantity = Column(Integer)
    order_date = Column(DateTime, default=datetime.now)
    # define idx_order_id as table index
    __table_args__ = (Index('idx_order_id', 'order_id'),)

    # Define relationships
    customer = relationship("Customer", back_populates="orders")
    product = relationship("Product", back_populates="orders")

# Define relationships for Customer and Product models
Customer.orders = relationship("Order", order_by=Order.order_id, back_populates="customer")
Product.orders = relationship("Order", order_by=Order.order_id, back_populates="product")


## Generate Database

#### Using SQLite

In [ ]:
engine_sqlite = create_engine('sqlite:///ecommerce.db', echo=True, future=True)
Base.metadata.create_all(engine_sqlite)

#### Using SQL Server

In [ ]:
import urllib

params = urllib.parse.quote_plus(f'DRIVER=ODBC Driver 17 for SQL Server;SERVER={server};DATABASE={database};UID={username};PWD={password}')

engine_mssql = create_engine(f'mssql+pyodbc:///?odbc_connect={params}', echo=True, future=True)

Base.metadata.create_all(engine_mssql)

In [ ]:
#  # Load credentials
# from cryptography.fernet import Fernet
# import json

# # Secret Key and Credential Loading
# with open('secret.key', 'rb') as key_file:
#     key = key_file.read()

# fernet = Fernet(key)

# # Load and decrypt the credentials
# with open('credentials.encrypted', 'rb') as encrypted_file:
#     encrypted_credentials = encrypted_file.read()
#     decrypted_credentials = fernet.decrypt(encrypted_credentials)

# # Convert back from JSON
# credentials = json.loads(decrypted_credentials.decode())

# # Access credentials
# server = credentials['server']
# database = credentials['database']
# username = credentials['username']
# password = credentials['password']

## Ingest Data to SQL

In [ ]:
def add_data_to_database(session, customers, products, orders):
    # Add customers
    for idx, row in customers.iterrows():
        customer = Customer(customer_id=row['customer_id'], name=row['name'], email=row['email'])
        session.add(customer)

    # Add products
    for idx, row in products.iterrows():
        product = Product(product_id=row['product_id'], name=row['name'], price=row['price'], category=row['category'])
        session.add(product)

    # Add orders
    for idx, row in orders.iterrows():
        order = Order(order_id=idx + 1, customer_id=row['customer_id'], product_id=row['product_id'], quantity=row['quantity'], order_date=row['order_date'])
        session.add(order)

    session.commit()
    
    
with Session(engine_sqlite) as session:
    add_data_to_database(session, customers, products, orders)

with Session(engine_mssql) as session:
    add_data_to_database(session, customers, products, orders)

## Extract Data from SQLAlchemy

In [ ]:
# Extract data from SQL
def fetch_data():
    stmt = select(Order.order_id, Order.quantity, Order.order_date,
                  Product.name.label('product_name'), Product.price,
                  Customer.name.label('customer_name')).\
        join(Product, Order.product_id == Product.product_id).\
        join(Customer, Order.customer_id == Customer.customer_id)

    results = session.execute(stmt).fetchall()
    session.close()

    data = pd.DataFrame(results, columns=['order_id', 'quantity', 'order_date', 'product_name', 'price', 'customer_name'])
    return data

data = fetch_data()

In [ ]:
data.head()

In [ ]:
# Calculate total sales per product
data['total_sales'] = data['quantity'] * data['price']
sales_per_product = data.groupby('product_name')['total_sales'].sum().sort_values(ascending=False)

In [ ]:
import matplotlib.pyplot as plt

# Plot total sales per product
plt.figure(figsize=(12, 6))
sales_per_product.plot(kind='bar', color='skyblue')
plt.title('Total Sales Per Product')
plt.xlabel('')
plt.xticks(rotation=45)
plt.ylabel('Total Sales')
plt.show()


## Ending

### SQLAlchemy Architecture

<div style="text-align:center;">
    <img src="SQLAlchemy_Architecture.png" alt="SQLAlchemy_Architecture" style="display:block; margin:auto;" width="400"/>
    <p style="text-align: center;">Credit: <a href="link_to_source">The Architecture of SQLAlchemy</a></p>
</div>


### Sources
- [SQLAlchemy 2.0 Documentation](https://docs.sqlalchemy.org/en/20/)
- [Creating a database using Python and SQLAlchemy](https://medium.com/@sandyjtech/creating-a-database-using-python-and-sqlalchemy-422b7ba39d7e)


### Created BY: [![LinkedIn](https://img.shields.io/badge/-Ziyad_Alshawi-blue?style=flat&logo=LinkedIn)](https://www.linkedin.com/in/zalshawi)